# Lession 5 - 文本内容转换

对于LLM来说，它非常擅长将输入内容转换成不同的格式，实现例如文本翻译、拼写及语法纠正、语气调整以及格式转换等任务。

值得注意的是，ChatGPT在回答问题时，并非先将中文翻译为英文，再根据英文去寻找答案。ChatGPT自身支持多语言，在接收到中文提示词时，ChatGPT会使用中文相关数据进行回答；收到英文提示词时，则是基于英文相关数据进行回答。回答的质量取决于模型训练数据的质量。

In [7]:
import openai
import os
import time

os.environ["http_proxy"] = "http://127.0.0.1:<端口号>"
os.environ["https_proxy"] = "http://127.0.0.1:<端口号>"

openai.api_key  = 'sk-'

def get_completion(prompt, model="gpt-3.5-turbo"):
    '''
    prompt: 对应的提示
    model: 调用的模型，默认为 gpt-3.5-turbo(ChatGPT)，有内测资格的用户可以选择 gpt-4
    '''
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # 模型输出的温度系数，控制输出的随机程度
    )
    # 调用 OpenAI 的 ChatCompletion 接口
    return response.choices[0].message["content"]

## 1. 文本翻译与语种识别

### 1.1 语种识别

In [4]:
prompt = f"""
请告诉我以下文本是什么语种:
```Combien coûte le lampadaire?```
"""
response = get_completion(prompt)
print(response)

这是法语。


### 1.2 多语种通用翻译器

In [12]:
user_messages = [
  "La performance du système est plus lente que d'habitude.",  # System performance is slower than normal
  "Mi monitor tiene píxeles que no se iluminan.",              # My monitor has pixels that are not lighting
]

In [13]:
for issue in user_messages:
    time.sleep(20)
    prompt = f"告诉我以下文本是什么语种，直接输出语种，如法语，无需输出标点符号: ```{issue}```"
    lang = get_completion(prompt)
    print(f"原始消息 ({lang}): {issue}\n")

    prompt = f"""
    将以下消息分别翻译成英文和中文，并写成
    中文翻译：xxx
    英文翻译：yyy
    的格式：
    ```{issue}```
    """
    response = get_completion(prompt)
    print(response, "\n=========================================")

原始消息 (法语): La performance du système est plus lente que d'habitude.

中文翻译：系统性能比平时慢。
英文翻译：The system performance is slower than usual. 
原始消息 (西班牙语): Mi monitor tiene píxeles que no se iluminan.

中文翻译：我的显示器有一些像素点不亮。
英文翻译：My monitor has pixels that don't light up. 


## 2. 语气调整

除了实现文本翻译外，LLM还能对语气风格进行调整，例如在与朋友聊天时采用非正式语气，在工作时采用正式化的书面用词。

In [15]:
prompt = f"""
将以下文本翻译成商务信函的格式:
```哎，上回你说咱部门要采购的显示器是多少寸来着？```
"""
response = get_completion(prompt)
print(response)

尊敬的XXX，

您好！我想确认一下我们部门需要采购的显示器尺寸是多少寸？感谢您的帮助！

谢谢！

此致

敬礼

XXX


In [18]:
prompt = f"""
将以下文本改用朋友间聊天时非正式的随意口气进行描述:
```尊敬的XXX，

您好！我想确认一下我们部门需要采购的显示器尺寸是多少寸？感谢您的帮助！

谢谢！

此致

敬礼

XXX```
"""
response = get_completion(prompt)
print(response)

嘿，老铁！

你好啊！我想问一下我们部门要买啥尺寸的显示器啊？谢谢你啦！

拜拜！


## 3. 格式转换

在实际场景中，LLM可以轻松将结构化的内容转换成另一种结构信息，实现从JSON到HTML、XML以及MarkDown的格式转换

In [19]:
data_json = { "resturant employees" :[
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

In [20]:
prompt = f"""
将以下Python字典从JSON转换为MarkDown表格，保留表格标题和列名：{data_json}
"""
response = get_completion(prompt)
print(response)

| resturant employees | name | email |
| --- | --- | --- |
|  | Shyam | shyamjaiswal@gmail.com |
|  | Bob | bob32@gmail.com |
|  | Jai | jai87@gmail.com |


## 4.拼写及语法纠正

拼写及语法的检查与纠正是一个十分常见的需求，特别是使用非母语语言，例如发表英文论文时，这是一件十分重要的事情。
以下给了一个例子，有一个句子列表，其中有些句子存在拼写或语法问题，有些则没有，我们循环遍历每个句子，要求模型校对文本，如果正确则输出“未发现错误”，如果错误则输出纠正后的文本。

In [22]:
text = f"""
Got this for my daughter for her birthday cuz she keeps taking \
mine from my room.  Yes, adults also like pandas too.  She takes \
it everywhere with her, and it's super soft and cute.  One of the \
ears is a bit lower than the other, and I don't think that was \
designed to be asymmetrical. It's a bit small for what I paid for it \
though. I think there might be other options that are bigger for \
the same price.  It arrived a day earlier than expected, so I got \
to play with it myself before I gave it to my daughter.
"""

In [23]:
prompt = f"校对并更正以下商品评论：```{text}```"
response = get_completion(prompt)
print(response)

I got this for my daughter's birthday because she keeps taking mine from my room. Yes, adults also like pandas too. She takes it everywhere with her, and it's super soft and cute. However, one of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. It's also a bit smaller than I expected for the price. I think there might be other options that are bigger for the same price. On the bright side, it arrived a day earlier than expected, so I got to play with it myself before giving it to my daughter.


In [24]:
from redlines import Redlines
from IPython.display import display, Markdown

diff = Redlines(text,response)
display(Markdown(diff.output_markdown))

<span style="color:red;font-weight:700;text-decoration:line-through;">Got </span><span style="color:red;font-weight:700;">I got </span>this for my <span style="color:red;font-weight:700;text-decoration:line-through;">daughter for her </span><span style="color:red;font-weight:700;">daughter's </span>birthday <span style="color:red;font-weight:700;text-decoration:line-through;">cuz </span><span style="color:red;font-weight:700;">because </span>she keeps taking mine from my <span style="color:red;font-weight:700;text-decoration:line-through;">room.  </span><span style="color:red;font-weight:700;">room. </span>Yes, adults also like pandas <span style="color:red;font-weight:700;text-decoration:line-through;">too.  </span><span style="color:red;font-weight:700;">too. </span>She takes it everywhere with her, and it's super soft and <span style="color:red;font-weight:700;text-decoration:line-through;">cute.  One </span><span style="color:red;font-weight:700;">cute. However, one </span>of the ears is a bit lower than the other, and I don't think that was designed to be asymmetrical. It's <span style="color:red;font-weight:700;">also </span>a bit <span style="color:red;font-weight:700;text-decoration:line-through;">small </span><span style="color:red;font-weight:700;">smaller than I expected </span>for <span style="color:red;font-weight:700;text-decoration:line-through;">what I paid for it though. </span><span style="color:red;font-weight:700;">the price. </span>I think there might be other options that are bigger for the same <span style="color:red;font-weight:700;text-decoration:line-through;">price.  It </span><span style="color:red;font-weight:700;">price. On the bright side, it </span>arrived a day earlier than expected, so I got to play with it myself before <span style="color:red;font-weight:700;text-decoration:line-through;">I gave </span><span style="color:red;font-weight:700;">giving </span>it to my daughter.